In [1]:
#find maximal cliques from the graph constructed using tweet format

#steps
#
#1: read from tweet file 
#2: identify hashtags, users, sentiments
#3: build clique edgelist from combinations of elements within each tweet
#4: use networkx to find cliques, find maximal cliques

def load_sentiments(filename):  
    sentiments = []
    lines = open(filename)
    for line in lines:
        sentiments.append(line.lstrip().rstrip())
    return sentiments

In [2]:
from itertools import combinations
import networkx as nx
from nltk.collocations import BigramCollocationFinder
from nltk.corpus import stopwords
from nltk.metrics import BigramAssocMeasures


In [3]:
relevent_tweets = open("twitterout.txt", encoding = 'utf-8')
unrestricted_tweets = relevent_tweets.readlines()

def create_usr_tweet_tuples(tweets):
    tweet_tuples = []
    for tweet in tweets:
        if tweet is not None:
            temp_tuple = tweet.split("|")
            if len(temp_tuple) == 2:
                tweet_tuples.append((temp_tuple[0], temp_tuple[1]))
            #print(tuples)
    return tweet_tuples

tweets = create_usr_tweet_tuples(unrestricted_tweets)

In [4]:
def find_bigrams(tweets):
    
    tweet_text = []
    twitter_bigrams = []

    for tweet in tweets:
        tweet_text = tweet[0] + tweet[1]
    
    #object to operate on list of words

    finder = BigramCollocationFinder.from_words(tweet_text.split())

    ignored_words = set(stopwords.words('english'))

    filter_stops = lambda ignored_word: len(ignored_word) < 4 or ignored_word in ignored_words

    finder.apply_word_filter(filter_stops)
    
    #number of bigrams
    list_of_bigrams = finder.nbest(BigramAssocMeasures.likelihood_ratio, 100)

    #finds bigrams not in ignored
    
    #bigram= []
    #print(list_of_bigrams)
    
    for tweet in tweets:
        for bigram in list_of_bigrams:
            bigram_text = bigram[0] + " " + bigram[1]
            if tweet[1].find(bigram_text) > 0:
                twitter_bigrams.append((tweet[0], bigram_text))
           
    return twitter_bigrams

In [15]:
sentiments = load_sentiments('Sentiments2.txt')

bigrams = find_bigrams(tweets)

heterogenous_list = []
    
for tweet in tweets:
    #string = line
    line = tweet[0] + ' ' + tweet[1]
    line = line.replace('|', ' ')
    tweet_elements = []
    tokens = line.split()
    #print(bigram)

    for token in tokens: #checking if this token starts with #, @, or is a sentiment
        if token.startswith('#'):
            #print("hashtag found: ", token)
            tweet_elements.append(token)
        elif token.startswith('@'):
            #print("user found: ", token)
            tweet_elements.append(token)
        elif token in sentiments:
            #print("sentiment found: ", token)
            tweet_elements.append(token)
        else:
            pass
    for bigram in bigrams:
        if line.find(bigram[1]) > 0:
            tweet_elements.append(bigram[1])
        
    #step 3, builds clique edgelist from tweet elements for each tweet
    combs = combinations(tweet_elements, 2)
    for comb in combs:
        heterogenous_list.append(comb)
print(heterogenous_list)

[('@innovate_k', '@scottlenet:'), ('@innovate_k', 'great'), ('@innovate_k', '@selinatroesch'), ('@scottlenet:', 'great'), ('@scottlenet:', '@selinatroesch'), ('great', '@selinatroesch'), ('@tomgilks', '@theofleury14:'), ('@tomgilks', 'good'), ('@theofleury14:', 'good'), ('@belicia_campas', '@quepasadulce'), ('@belicia_campas', '@deassassins2020'), ('@belicia_campas', 'good'), ('@quepasadulce', '@deassassins2020'), ('@quepasadulce', 'good'), ('@deassassins2020', 'good'), ('@cusic_steve', '@gop'), ('@cusic_steve', 'great'), ('@gop', 'great'), ('@___madisonrose_', 'happy'), ('@yasminecalles', '@hpoutx:'), ('@yasminecalles', 'great'), ('@hpoutx:', 'great'), ('@alfargo58', '@aaronbastani'), ('@alfargo58', '@ajsislington'), ('@alfargo58', '@bbcquestiontime'), ('@alfargo58', '@ayocaesar'), ('@aaronbastani', '@ajsislington'), ('@aaronbastani', '@bbcquestiontime'), ('@aaronbastani', '@ayocaesar'), ('@ajsislington', '@bbcquestiontime'), ('@ajsislington', '@ayocaesar'), ('@bbcquestiontime', '@ayo

In [12]:
#constructing heterogenous graph

H = nx.Graph()
H.add_edges_from(heterogenous_list)

In [13]:
def topk_list(filename):
    listfile = open(filename)
    rawlist = listfile.readlines()
    topk = []
    for line in rawlist:
        if line is not None:
            line = line.replace('\n', '')
            topk.append(line)
    return topk 

topsentiments = topk_list("Playground1.txt")
topbigrams = topk_list("Playground2.txt")
tophashtags = topk_list("Playground3.txt")
topmentions = topk_list("PLayground4.txt")

In [19]:
cliques = nx.enumerate_all_cliques(H)
        
mcs = nx.find_cliques(H) #getting maximal cliques

print(topmentions)
positive_cliques = []

for mc in mcs:
    if len(mc) >= 2:
        #print(mc)
        #print("Length of clique => ", len(mc), " and clique is ", mc)
        for element in mc:
#             if element in topbigrams:
#                 positive_cliques.append(mc)
#             if element in topsentiments:
#                 positive_cliques.append(mc)
#             if element in tophashtags:
#                 positive_cliques.append(mc)
            if element in topmentions:
                positive_cliques.append(mc)
            
for element in positive_cliques:
    print(element)
#print(nx.info(H))

['#account', '#development', '#dallastexasunitedstates,', '#ragancontent.', '#thirdgradeteachersrock', '#câ€¦', '#operations', '#lead', '#sandiego', '#catalent.', '#vaping', '#vapingban', '#truth', '#sacramento,', '#tcrwp', '#cesleads', '#kellerliteracy', '#deepthhroat', '#workingtogether', '#roughsex', '#donlemonhypocrite', '#notajournalist', '#opinionisnotfact', '#modernwarfarebetacodes', '#modernwarfarebetacode', '#betamodernwarfare', '#betacode', '#westvancouver,', '#ttp', '#championshipculture']
['@tmj_sac_labor', '#sacramento,', '#labor']
['#westvancouver,', '@westvanrt', '#retail', '@tmj_van_jobs:']
['@us_fda', '@cdcgov', '@secazar', '#vaping', '#truth', 'good', '@jennym_vs', '@realdonaldtrump', '#vapingban']
['@us_fda', '@cdcgov', '@secazar', '#vaping', '#truth', 'good', '@jennym_vs', '@realdonaldtrump', '#vapingban']
['@us_fda', '@cdcgov', '@secazar', '#vaping', '#truth', 'good', '@jennym_vs', '@realdonaldtrump', '#vapingban']
['#donlemonhypocrite', '#notajournalist', '#opinio